In [139]:
import finite_differences.finite_diff_formulas as f
import importlib as i
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve
i.reload(f)
import test_fd_methods as t
i.reload(t)
import test_implementation as oi
i.reload(oi)

<module 'test_implementation' from '/Users/manuelpanora/Python/Finite-Difference-Option-Pricer/test_implementation.py'>

# Option Pricing with the Crank-Nicolson Method

## Overview
This notebook demonstrates pricing European call options using methods of PDE solutions. The two main methods explored are the Crank-Nicolson method and forward finite differnces. The goal of this project is to price exotic options using similar numerical methods. 

**Key Features**:
- Finite difference method with dynamic grid sizing.
- Sparse matrix solving for efficiency.
- Comparison to Black-Scholes analytical solution.
- Rigorous Testing to improve stability and accuracy

The parameters below are used for intial method testing. The analytical method and the naive finite difference method are displayed first.

In [103]:
TRADING_DAYS = 252
K = 95
S = 100
sigma = .2
r = .05
tfinal = 21 / TRADING_DAYS  

#n is discrete time steps, m is assset price steps
n = 16
m = 16        

bs_price = f.black_scholes_call(S, K, tfinal, r, sigma)
print("European Call Option Price Analytical:", bs_price)

European Call Option Price Analytical: 5.898921658718436


In [104]:
price_fd = f.finite_difference_call(n, m, K, tfinal, S, sigma, r)
price_cn = f.price_derivative_cn(n, m, K, r, tfinal, S, sigma)


79.89168748233948 112.96544464647876


With the completed implementations of both methods, we can display the results. We observe that given our starting conditions, both numerical methods get within one digit of accuracy.

In [105]:
print("When given", m, "pricesteps and", n, "timesteps:")
print("European Call Option Price Analytical:", bs_price)
print("European Call Option Price Finite Dif:", price_fd)
print("European Call Option Price Crank-Nico:", price_cn)

When given 16 pricesteps and 16 timesteps:
European Call Option Price Analytical: 5.898921658718436
European Call Option Price Finite Dif: 5.876594250948936
European Call Option Price Crank-Nico: 5.296782858142622


We continue to analyze the convergence of the solution. Analyzing and finding the source of innacuracy is the primary focus of attention. Using an online source as a method of validation, we compute the option price for varying asset and time step values. As the rsults show, when we simply increase the number of timesteps or asset price steps, it is clear that increasing the asset price steps influences the convergence of the solution more. 

In [132]:
#When did the online implementation become so good? Make sure to compile everything? This is so strange. I need to understand what is happening here. THis must be a mistake
cn_errors, oi_errors, cn_values, neumann = t.test_convergence_all(K, r, tfinal, S, sigma, (5,4), f.price_derivative_cn, oi.crank_nicholson)

print(cn_errors)
print(oi_errors)

UnboundLocalError: cannot access local variable 'dels' where it is not associated with a value

We will focus on creating the optimal grid that results in the smallest errors. The test is stil conducted using various asset price step sizes. Empircal testing shows that 3 standard deviations used to calculate the boundary values for the asset price grid are sufficient and yield the best numerical results in one particular case. Testing is provided below. With this method, we have narrowed down the range of standard deviations to (2,5) as these ranges provide best convergence and numerical stability. For additional testing, we expriment with varying starting values. (Best combination so far (4,2,3))

In [8]:
def dynamic_bounds(K, sigma, T, n_std=3):
    """
    Generate asset price bounds based on lognormal distribution percentiles.
    """
    S_min = K * np.exp(-n_std * sigma * np.sqrt(T))
    S_max = K * np.exp(n_std * sigma * np.sqrt(T))
    return (S_min, S_max)

In [93]:
#Static boundaries: 
s_vals = [(K/3, K*2), (K/4, K*3)]

#Dynamic boundaries:
d_vals = np.array([dynamic_bounds(K, sigma, tfinal, i) for i in range(2,5)])

cn_errors, oi_errors, neumann = t.test_grid_asset_boundaries(f.price_derivative_cn, oi.crank_nicholson, K, r, tfinal, S, sigma, (6,5), d_vals)

for i in range(len(cn_errors)):
    print(d_vals[i])
    print(cn_errors[i])

[ 224.66448206 1112.77046425]
[[ 0.          0.          0.          0.          0.        ]
 [48.71651345  0.          0.          0.          0.        ]
 [51.00739688  0.          0.          0.          0.        ]
 [51.80982537 19.50291643  0.          0.          0.        ]
 [52.03093995 19.7689004   8.67902374  0.          0.        ]
 [52.08763813 19.83711838  8.67650921  0.          0.        ]]
[ 150.59710596 1660.05846137]
[[ 0.          0.          0.          0.          0.        ]
 [25.48802199  0.          0.          0.          0.        ]
 [26.0256248   0.36562828  0.          0.          0.        ]
 [26.2162442   0.04164386  0.          0.          0.        ]
 [26.26898685  0.15438988  0.71221788  0.          0.        ]
 [26.28252629  0.18333618  0.71793987  0.          0.        ]]
[ 100.948259  2476.5162122]
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [5.98301621e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000

Our next step is to test various starting conditions to see which these work best for. Our testing shows that this method works best for low volatility, short time to maturity. Generally speaking, 3 tends to produce the best results out of any even though some of the boundary conditions are not as good as others. 

In [13]:
# Test cases: K, σ, T, S0
test_cases = [
    # Low volatility, short maturity
    {"K": 100, "sigma": 0.1, "T": 0.1, "S0": 100, "type" : "Low volatility, short maturity"},
    # High volatility, long maturity
    {"K": 100, "sigma": 0.5, "T": 5.0, "S0": 100, "type" : "High volatility, long maturity"},
    # Extreme moneyness (Strike price is significantly higher/lower than current stock price )
    {"K": 100, "sigma": 0.3, "T": 1.0, "S0": 50, "type" : "OTM"},   # OTM call
    {"K": 100, "sigma": 0.3, "T": 1.0, "S0": 150, "type" : "ITM"},  # ITM call
    # Small/large strikes
    {"K": 10,  "sigma": 0.2, "T": 1.0, "S0": 10, "type" : "Small strike"},
    {"K": 500, "sigma": 0.4, "T": 1.0, "S0": 500, "type" : "Large strike"},
]

for case in test_cases:
    K = case["K"]
    sigma = case["sigma"]
    tfinal = case["T"]
    S0 = case["S0"]
    
    d_vals = np.array([dynamic_bounds(K, sigma, tfinal, i) for i in range(2,5)])

    cn_errors, oi_errors, conds = t.test_grid_asset_boundaries(f.price_derivative_cn, oi.crank_nicholson, K, r, tfinal, S0, sigma, (6,5), d_vals)
    print(case["type"])
    for i in range(len(cn_errors)):
        print(d_vals[i])
        print(cn_errors[i])
    
    print()

Low volatility, short maturity
[ 93.87129414 106.52883921]
[[0.77532033 0.         0.         0.         0.        ]
 [0.6251313  0.15707062 0.         0.         0.        ]
 [0.52990449 0.05566836 0.         0.         0.        ]
 [0.49620838 0.01983332 0.13348973 0.         0.        ]
 [0.48689165 0.00993453 0.12515464 0.         0.        ]
 [0.48450045 0.00739469 0.12301753 0.53875701 0.        ]]
[ 90.94926797 109.95140724]
[[7.96495992e-01 2.71301042e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [6.73389897e-01 1.27496312e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [5.93299255e-01 3.38308652e-02 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [5.64560812e-01 2.28199627e-04 1.05681339e-01 0.00000000e+00
  0.00000000e+00]
 [5.56577120e-01 9.10501084e-03 9.69965742e-02 0.00000000e+00
  0.00000000e+00]
 [5.54525393e-01 1.15033987e-02 9.47676802e-02 4.05117630e-01
  0.00000000e+00]]
[ 88.11819864 113.48393583]
[[0.69705782 0.20390168 0.         0.         0.       

Due to the variation in the intial conditions, and the size of the boundaries, we shoudl conclude that the boundaries should be adaptive to the sizes we are willing to take. Or we should take step sizes that are adapted to the original conditions, regardless we must satisfy this ineqaulity: $\Delta t \leq \frac{(\Delta S)^2}{2\sigma^2S_{max}^2}$

We will begin by testing to see when this condition holds and when it is violated and by how much. We will use a narrower boundary for the standard devations, ultimately choosing just 3 standard deviations for each case

In [140]:
for case in test_cases:
    K = case["K"]
    sigma = case["sigma"]
    tfinal = case["T"]
    S0 = case["S0"]
    
    c = 3.5 if sigma*np.sqrt(tfinal) > 1 else 3
    d_vals = dynamic_bounds(K, sigma, tfinal, c)

    cn_errors, oi_errors, vals, conds = t.test_convergence_all(K, r, tfinal, S0, sigma, (6,5), f.price_derivative_cn, oi.crank_nicholson, True, d_vals)
    print(case["type"])
    
    print(cn_errors)    
    print()

6.3340464233070195 0.03333333333333333
True
3 3

0.7964959920104118
3.1670232116535098 0.03333333333333333
True
6 3

0.27130104155702517
1.0556744038845034 0.03333333333333333
False
18 3

0.28791120105941 0.03333333333333333
False
66 3

0.07365170259659326 0.03333333333333333
False
258 3

6.3340464233070195 0.016666666666666666
True
3 6

0.6733898969494675
3.1670232116535098 0.016666666666666666
True
6 6

0.12749631208189594
1.0556744038845034 0.016666666666666666
False
18 6

0.28791120105941 0.016666666666666666
False
66 6

0.07365170259659326 0.016666666666666666
False
258 6

6.3340464233070195 0.005555555555555556
True
3 18

0.5932992550711726
3.1670232116535098 0.005555555555555556
True
6 18

0.033830865194159676
1.0556744038845034 0.005555555555555556
False
18 18

0.28791120105941 0.005555555555555556
False
66 18

0.07365170259659326 0.005555555555555556
False
258 18

6.3340464233070195 0.0015151515151515152
True
3 66

0.5645608122177842
3.1670232116535098 0.0015151515151515152
Tr

Generally, we would like to maximize the accuracy of NAS and NTP when creating the grid. This is the motivation for the next step. As demonstrated here, the advection condition holds in almost every case. Still, this has confirmed that in every case, enforcing this condition does not reduce our ability to find viable solutions.

In [26]:
#Define function so that for each case in the starting conditions, we check whether or not the n and m satisfy the advection conditions. 
#If it does and the results are consistent meaning they have the best accuracy, then we should implement it otherwise we won't

#For each test case
for case in test_cases:
    K = case["K"]
    sigma = case["sigma"]
    tfinal = case["T"]
    S0 = case["S0"]

    cn_errors = t.satisfies_advection_term(K, r, tfinal, S0, sigma, (6,5), f.price_derivative_cn)
    print(case["type"])
    print(cn_errors)
    print()
    #Call function returning accuracy of option pricer and whether or not it satisfies the avection conditions

    

Low volatility, short maturity
[[7.96495992e-01 2.71301042e-01 3.84692139e-01 6.38280558e-01
  1.12703224e+00]
 [6.73389897e-01 1.27496312e-01 2.25931402e-01 4.28716189e-01
  1.02374892e+00]
 [5.93299255e-01 3.38308652e-02 1.37096832e-01 4.22303521e-01
  1.09799822e+00]
 [5.64560812e-01 2.28199627e-04 1.05681339e-01 4.09505523e-01
  1.11739413e+00]
 [5.56577120e-01 9.10501084e-03 9.69965742e-02 4.06010153e-01
  1.12074835e+00]
 [5.54525393e-01 1.15033987e-02 9.47676802e-02 4.05117630e-01
  1.12160473e+00]]

High volatility, long maturity
[[ 0.          0.          0.         32.54123956 56.57558089]
 [ 0.          0.         10.61099683 37.38930136 49.29184223]
 [ 0.         27.07269669  4.06130475 39.17908364 49.55813723]
 [46.04623363 25.74796246  1.88514626 39.75917472 49.563965  ]
 [46.05794596 25.38855481  1.29253842 39.91581469 49.56573847]
 [46.06091702 25.29675146  1.14102093 39.9557772  49.56619938]]

OTM
[[0.00000000e+00 4.30319595e-01 8.10910490e-02 1.10223993e-01
  1.173277

There are many directions from here. As we have not found any condition that is sufficient to distinguish NAS and NTS, I have found two plausible directions, the second of which builds on the first one. The first ideas is to have a lookup table where given the initial conditions, we lookup the optimal NAS and NTS, but a different direction taken here is to use this lookup table to generate a continuous mapping of the initial conditions onto (NAS, NTS) and its main advantage is that with this continous mapping, we will generate the optimal NAS and NTS for points not seen before. Interest rate is assumed to be constant.

In [121]:
#This will be placed in another file, used for generating the ideal function.

#K_vals initialized to focus on values around 50 - 150
K_vals = np.round(np.exp(np.linspace(np.log(50), np.log(500), 30)).astype(int))

#More precision for short term, we concatenate 
T_vals = np.concatenate([
    np.linspace(0.01, 0.1, 15),    # Short-term
    np.linspace(0.1, 1.0, 10),     # Medium
    np.linspace(1.0, 5.0, 5)       # Long-term
])

#Cluster the initial s values around the K values
#Something to think about (How much emphasis should we place on unlikely combinations?)
S_vals = np.unique(np.concatenate([np.linspace(0.5 * K, 1.5 * K, 20) for K in K_vals]))

#Round the volatilies to 2 digits as that is what is generally used
sigmas = np.round(np.linspace(0.1, 0.5, 10), 2)

#Initialize grid values to be tested on
param_grid = [(K, T, S, sigma) for K in K_vals for T in T_vals 
                  for S in S_vals if S for sigma in sigmas]

#For each of these values, we are going to compute the optimal asset steps and price steps and then we are going to store the pair to be used later
#or i in range(len(param_grid)//10000):
    #print(t.optimal_NAS_NTS(*(param_grid[i]), r))

#K = 95
#S = 100
#sigma = .2
#r = .05
#tfinal = 21 / TRADING_DAYS  
print(t.optimal_NAS_NTS(95, 21/252, 100, .2, .05))



32 32
1.0335549113793525 0.0026041666666666665
342 32
1.0335549113793525 0.0009259259259259259
483 32
1.0335549113793525 0.0006775067750677507
591 32
1.0335549113793525 0.0005555555555555556
683 32
1.0335549113793525 0.00048449612403100775
763 32
1.0335549113793525 0.00043402777777777775
836 32
1.0335549113793525 0.00039872408293460925
903 32
1.0335549113793525 0.0003687315634218289
965 32
1.0335549113793525 0.0003457814661134163
1024 32
1.0335549113793525 0.0003255208333333333
32 90
0.09670689229280491 0.0026041666666666665
342 90
0.09670689229280491 0.0009259259259259259
483 90
0.09670689229280491 0.0006775067750677507
591 90
0.09670689229280491 0.0005555555555555556
683 90
0.09670689229280491 0.00048449612403100775
763 90
0.09670689229280491 0.00043402777777777775
836 90
0.09670689229280491 0.00039872408293460925
903 90
0.09670689229280491 0.0003687315634218289
965 90
0.09670689229280491 0.0003457814661134163
1024 90
0.09670689229280491 0.0003255208333333333
32 123
0.068475687710433